# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from sklearn.model_selection import train_test_split

## Dataset

### Overview
We wil predict median house valie from kaggle's data set "Calofornia housing". Data set was downloaded as a csv and uploaded i.e. registered as data set in the workspace. 


### Workspace Set Up and Initialize an Experiment

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RFXLTBC76 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-127655
aml-quickstarts-127655
southcentralus
fb968fd6-afa4-4fb2-8296-1d0120d715b4


In [3]:
os.listdir(os.curdir), os.getcwd()

(['.ipynb_aml_checkpoints',
  'automl (2).ipynb',
  'housing.csv',
  'raw_data.json',
  'scoring.py'],
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook127655/code/Users/odl_user_127655')

In [4]:
# choose a name for experiment
experiment_name = 'automl_housing'
project_folder = './automl-housing'
os.makedirs(project_folder, exist_ok=True)

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl_housing,quick-starts-ws-127655,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach a Compute Resource

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "notebook127655"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


### Data set

In [7]:
import pandas as pd
data = pd.read_csv("housing.csv", header=0)

In [8]:
def prepare_data(data):    
    encoded_column=pd.get_dummies(data['ocean_proximity'], prefix='ocp')
    data=data.join(encoded_column)
    data=data.drop("ocean_proximity", axis=1)

    train, test = train_test_split(data, test_size=0.2, random_state=42)
    return train, test

train, test = prepare_data(data)
type(train), type(test), train.shape, test.shape

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 (16512, 14),
 (4128, 14))

In [9]:
columns=train.columns
train.to_csv(path_or_buf='trainset.csv', columns=columns, header=True, index=False) 
test.to_csv(path_or_buf='testset.csv', columns=columns, header=True, index=False)

In [10]:
datastore=ws.get_default_datastore() 
datastore.upload_files(['trainset.csv'])

Uploading an estimated of 1 files
Uploading trainset.csv
Uploaded trainset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [11]:
train=TabularDatasetFactory.from_delimited_files([(datastore, 'trainset.csv')])

datastore.upload_files(['testset.csv'])

Uploading an estimated of 1 files
Uploading testset.csv
Uploaded testset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [12]:
test=TabularDatasetFactory.from_delimited_files([(datastore, 'testset.csv')])

## AutoML Configuration

Since we are trying to predict a numeric value, in particular, "median_house_values", we have to define the task as "regression". Featurization is set to "auto" and we enable early stopping to avoid overfitting. We pass in our train and test sets and choose "normalized_mean_absolute_error" as the primary metric.

In [13]:
azureml.train.automl.utilities.get_primary_metrics("regression")

['normalized_root_mean_squared_error',
 'normalized_mean_absolute_error',
 'r2_score',
 'spearman_correlation']

In [14]:
# TODO: Put your automl settings here
# TODO: Put your automl config here

automl_setting={
    "featurization": "auto",
    "experiment_timeout_minutes": 30,
    "enable_early_stopping": True,
    "verbosity": logging.INFO,
    "compute_target": compute_target
}

task="regression" 
automl_config = AutoMLConfig( 
    task=task, 
    primary_metric='normalized_root_mean_squared_error', 
    training_data=train, 
    validation_data = test, 
    label_column_name='median_house_value', 
    **automl_setting
)

## Submit Run & Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
# TODO: Submit your experiment
from azureml.widgets import RunDetails

automl_run = experiment.submit(automl_config, show_output=True)

RunDetails(automl_run).show()

Running on remote.
Running on remote compute: notebook127655
Parent Run ID: AutoML_ab9d4fbb-757d-4bd6-bd50-644a4fe1ed05

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn 

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [16]:
automl_run.wait_for_completion()

{'runId': 'AutoML_ab9d4fbb-757d-4bd6-bd50-644a4fe1ed05',
 'target': 'notebook127655',
 'status': 'Completed',
 'startTimeUtc': '2020-11-24T06:17:20.070534Z',
 'endTimeUtc': '2020-11-24T06:48:54.757383Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook127655',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_housing","subscription_id":"fb968fd6-afa4-4fb2-8296-1d0120d715b4","resource_group":"aml-quickstarts-127655","workspace_name":"quick-starts-ws-127655","region":"southcentralus","compute_target":"notebook127655","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"normalized_root_mean_squared_error","task_type":"regression","data_script":null,"vali

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [17]:
# Retrieve and save your best automl model.
automl_run, fitted_automl_model = automl_run.get_output() 
print(fitted_automl_model) #To print
import joblib  
joblib.dump(fitted_automl_model, "automl_housing.pkl")

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

['automl_housing.pkl']

In [29]:
os.listdir(os.curdir), os.getcwd()

(['.azureml',
  '.config',
  '.ipynb_aml_checkpoints',
  'automl (2).ipynb',
  'automl-housing',
  'automl.log',
  'automl_housing.pkl',
  'azureml_automl.log',
  'housing.csv',
  'raw_data.json',
  'scoring.py',
  'testset.csv',
  'trainset.csv'],
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook127655/code/Users/odl_user_127655')

In [18]:
automl_run_metrics = automl_run.get_metrics()
automl_run_metrics

{'spearman_correlation': 0.9138951035775631,
 'normalized_root_mean_squared_error': 0.09934886736317816,
 'r2_score': 0.8228235321116265,
 'median_absolute_error': 21358.68757477193,
 'root_mean_squared_error': 48184.39936887613,
 'normalized_root_mean_squared_log_error': 0.06811054968772484,
 'normalized_mean_absolute_error': 0.06675515129322772,
 'mean_absolute_percentage_error': 18.38221463878435,
 'mean_absolute_error': 32376.38188751803,
 'root_mean_squared_log_error': 0.23883385833995038,
 'explained_variance': 0.8228777634621539,
 'normalized_median_absolute_error': 0.04403834948056282,
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_ab9d4fbb-757d-4bd6-bd50-644a4fe1ed05_0/predicted_true',
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_ab9d4fbb-757d-4bd6-bd50-644a4fe1ed05_0/residuals'}

In [19]:
print('Bets Run ID', automl_run.id)
print('\n MAE', automl_run_metrics['mean_absolute_error'])

Bets Run ID AutoML_ab9d4fbb-757d-4bd6-bd50-644a4fe1ed05_0

 MAE 32376.38188751803


In [21]:
from azureml.core.model import Model

model=Model.register(model_path="automl_housing.pkl", model_name="automl_housing", workspace=ws)

Registering model automl_housing


In [22]:
print(model.name, model.id)

automl_housing automl_housing:1


In [23]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
env = Environment(name="env")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")

# Adds dependencies to PythonSection of myenv
env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="scoring.py",
                                   environment=env)

In [24]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration()

In [30]:
from azureml.core.model import Model

model = Model(ws, name='automl_housing')
service = Model.deploy(ws, 'myservice', [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 87db0648572145dbaab01b36a45128d2.azurecr.io
Logging into Docker registry 87db0648572145dbaab01b36a45128d2.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 87db0648572145dbaab01b36a45128d2.azurecr.io/azureml/azureml_dc785bef270e2bc43828ea6503b50a5e
 ---> cf3bff41dfae
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> c84f75cd18e3
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImZiOTY4ZmQ2LWFmYTQtNGZiMi04Mjk2LTFkMDEyMGQ3MTViNCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEyNzY1NSIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEyNzY1NSIsIndvcmtzcGFjZUlkIjoiODdkYjA2NDgtNTcyMS00NWRiLWFhYjAtMWIzNmE0NTEyOGQyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in fc3bd807c60d
 ---> 47aecb6e565f
Step 4/5 : RUN mv '/var/azureml-app/tmp0oh1fk63.py' /var/azureml-app/main.py


ERROR - Error: Container has crashed. Did your init method fail?



WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [ ]:
test_set=pd.read_csv("testset.csv", header=0)
test_values=test_set.values
test_list=test_values.tolist()

In [ ]:
import json
test_data = json.dumps({"data": x_test.tolist()})
test_data = bytes(test_data, encoding = 'utf8')

result = service.run(input_data=test_data)

api_r2 = r2_score(y_test, np.array(result))
print(round(api_r2, 3))